# Mask-RCNN: mimic training on Coco Dataset

In [1]:
"""
written by wozhouh
"""

# Import Python Packages
import os
import imgaug

# Root directory of the project
ROOT_DIR = os.path.abspath("../")
import sys

# Import Mask RCNN
sys.path.append(ROOT_DIR)
from mrcnn import model as modellib
from optimization import mimicking

# Import COCO config
sys.path.append(os.path.join(ROOT_DIR, "samples", "coco"))  # To find local version
import coco

Using TensorFlow backend.


In [2]:
# Default setting
HOME_DIR = os.getenv('HOME')
DEFAULT_WEIGHTS_DIR = os.path.join(ROOT_DIR, "weights")
DEFAULT_LOGS_DIR = os.path.join(ROOT_DIR, "logs")
DEFAULT_DATASET_YEAR = "2017"
DEFAULT_COCO_PATH = os.path.join(HOME_DIR, "data", "Coco")

# weights to load
MODEL_PATH_UNDER_HOME = os.path.join(DEFAULT_WEIGHTS_DIR, "ResNet-101", "mask_rcnn_coco.h5")
INIT_MODEL_PATH = os.path.join(HOME_DIR, MODEL_PATH_UNDER_HOME)
LOG_DIR = os.path.join(DEFAULT_LOGS_DIR, "Mimicking")

In [3]:
# Mimicking config
class MimicConfig(coco.CocoConfig):
    NAME = "Mimicking"
    
    # GPU
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    GPU_COUNT = 2
    IMAGES_PER_GPU = 2
    
    # data
    IMAGE_MIN_DIM = 800
    IMAGE_MAX_DIM = 1024
    RPN_ANCHOR_SCALES = (32, 64, 128, 256, 512)
    
    # model
    TEACHER_BACKBONE = "resnet101"
    STUDENT_BACKBONE = "resnet50"
    BACKBONE_STRIDES = [4, 8, 16, 32, 64]
    
    # heads
    TOP_DOWN_PYRAMID_SIZE = 256
    RPN_TRAIN_ANCHORS_PER_IMAGE = 256
    
    # training
    LEARNING_RATE = 0.001
    WEIGHT_DECAY = 0.0001
    LOSS_WEIGHTS = {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'rpn_mimic_loss': 1.0}
    STEPS_PER_EPOCH = 1000
    VALIDATION_STEPS = 50
    
config = MimicConfig()
config.display()


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     4
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_HEAD                 original
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      2
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 2
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                93
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'rpn_mimic_loss': 1.0}
MASK_HEAD                      original
MASK_POOL_SIZE         

In [4]:
model = mimicking.MimicMaskRCNN(mode="training", model_dir=LOG_DIR, config=config)

In [5]:
# print the model summary
model.keras_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_image (InputLayer)        (None, None, None, 3 0                                            
__________________________________________________________________________________________________
input_image_meta (InputLayer)   (None, 93)           0                                            
__________________________________________________________________________________________________
input_rpn_match (InputLayer)    (None, None, 1)      0                                            
__________________________________________________________________________________________________
input_rpn_bbox (InputLayer)     (None, None, 4)      0                                            
__________________________________________________________________________________________________
lambda_3 (

In [6]:
# Load the weights
model.load_weights(INIT_MODEL_PATH, by_name=True)

In [7]:
# Load the dataset
train_dataset = coco.CocoDataset()
train_dataset.load_coco(DEFAULT_COCO_PATH, "train", year=DEFAULT_DATASET_YEAR)
train_dataset.prepare()
val_dataset = coco.CocoDataset()
val_dataset.load_coco(DEFAULT_COCO_PATH, "val", year=DEFAULT_DATASET_YEAR)
val_dataset.prepare()

# Image Augmentation
# Right/Left flip 50% of the time
augmentation = imgaug.augmenters.Fliplr(0.5)

loading annotations into memory...
Done (t=15.11s)
creating index...
index created!
loading annotations into memory...
Done (t=0.37s)
creating index...
index created!


In [ ]:
model.train(train_dataset=train_dataset,
                val_dataset=val_dataset,
                learning_rate=config.LEARNING_RATE,
                epochs=1,
                layers='all',
                augmentation=augmentation)


Starting at epoch 0. LR=0.001

Checkpoint Path: /home/processyuan/code/HumanMask/my-Mobile-Mask-RCNN/logs/Mimicking/mimicking20181211T1543/mask_rcnn_mimicking_{epoch:04d}.h5
Selecting layers to train
s_conv1                (Conv2D)
s_bn_conv1             (BatchNorm)
s_res2a_branch2a       (Conv2D)
s_bn2a_branch2a        (BatchNorm)
s_res2a_branch2b       (Conv2D)
s_bn2a_branch2b        (BatchNorm)
s_res2a_branch2c       (Conv2D)
s_res2a_branch1        (Conv2D)
s_bn2a_branch2c        (BatchNorm)
s_bn2a_branch1         (BatchNorm)
s_res2b_branch2a       (Conv2D)
s_bn2b_branch2a        (BatchNorm)
s_res2b_branch2b       (Conv2D)
s_bn2b_branch2b        (BatchNorm)
s_res2b_branch2c       (Conv2D)
s_bn2b_branch2c        (BatchNorm)
s_res2c_branch2a       (Conv2D)
s_bn2c_branch2a        (BatchNorm)
s_res2c_branch2b       (Conv2D)
s_bn2c_branch2b        (BatchNorm)
s_res2c_branch2c       (Conv2D)
s_bn2c_branch2c        (BatchNorm)
s_res3a_branch2a       (Conv2D)
s_bn3a_branch2a        (BatchN

/home/processyuan/anaconda3/envs/tflite/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:108: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/home/processyuan/anaconda3/envs/tflite/lib/python3.6/site-packages/keras/engine/training_generator.py:47: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the`keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'


Epoch 1/1
 659/1000 [==================>...........] - ETA: 11:15 - loss: 15.5479 - rpn_class_loss: 0.3764 - rpn_bbox_loss: 1.3919 - rpn_mimic_loss: 13.7796